In [1]:
!pip install -q -U peft

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training,LoraConfig,PeftModel,get_peft_model

from transformers import set_seed

set_seed(42)

2024-03-03 10:35:29.619270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 10:35:29.619369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 10:35:29.755072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token ('your-hf-token')"

In [4]:
model_path = "mistralai/Mistral-7B-Instruct-v0.2"

base_model = AutoModelForCausalLM.from_pretrained(
    model_path,  
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_path, 
                                          padding_side="left",
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

**Load the fineuned peft model, and set offload dir to prevent potential OOM error**

In [5]:
model = PeftModel.from_pretrained(base_model, "/kaggle/input/finetune-mistral-7b-peft-qlora-10-ep/mistral-finetuned-med-fp16/checkpoint-7000", offload_folder = "./offload/", device_map="auto")

In [6]:
model = model.merge_and_unload() # merge base model with peft 

***Create a Huggingface repo for uploading your model***

In [7]:
try:
    from huggingface_hub import create_repo

    create_repo("Nayal17/mistral-med-finetuned-fp16-10ep")
    
except:
    print("Repo already exists!")

In [8]:
hub_id = "mistral-med-finetuned-fp16-10ep" # repo name

## save model and tokenizer in local directory
model.save_pretrained(f"{hub_id}")
tokenizer.save_pretrained(f"{hub_id}")

('mistral-med-finetuned-fp16-10ep/tokenizer_config.json',
 'mistral-med-finetuned-fp16-10ep/special_tokens_map.json',
 'mistral-med-finetuned-fp16-10ep/tokenizer.model',
 'mistral-med-finetuned-fp16-10ep/added_tokens.json',
 'mistral-med-finetuned-fp16-10ep/tokenizer.json')

In [9]:
## push to hub
model.push_to_hub(f"{hub_id}", use_temp_dir=False)
tokenizer.push_to_hub(f"{hub_id}", use_temp_dir=False)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nayal17/mistral-med-finetuned-fp16-10ep/commit/f72c693e8e678499305a461893e6530824903a54', commit_message='Upload tokenizer', commit_description='', oid='f72c693e8e678499305a461893e6530824903a54', pr_url=None, pr_revision=None, pr_num=None)